## Setup paquetes y Google Drive

In [107]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import os
import time

## Descarga y preprocesamiento de datos

Hay cambios del notebook de partida (de encoding `utf-8` a `latin-1`).

In [108]:
texto = open("3_textos_literatura_española/La Celestina.txt", 'rb').read().decode(encoding='latin-1')
print('Longitud del texto:        {} carácteres'.format(len(texto)))

vocab = sorted(set(texto))

print ('El texto está compuesto de estos {} carácteres:'.format(len(vocab)))
print (vocab)

Longitud del texto:        376026 carácteres
El texto está compuesto de estos 90 carácteres:
['\n', ' ', '!', '"', '#', '&', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'X', 'Y', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'y', 'z', '{', '}', '~', '¡', '¿', 'Ü', 'á', 'ç', 'é', 'í', 'ñ', 'ó', 'ú']


### Procesamiento de los textos
#### Mapeo de caracteres

In [109]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

for char,_ in zip(char2idx, range(len(vocab))):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))

  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '"' :   3,
  '#' :   4,
  '&' :   5,
  '*' :   6,
  ',' :   7,
  '-' :   8,
  '.' :   9,
  '/' :  10,
  '0' :  11,
  '1' :  12,
  '2' :  13,
  '3' :  14,
  '4' :  15,
  '5' :  16,
  '6' :  17,
  '7' :  18,
  '8' :  19,
  '9' :  20,
  ':' :  21,
  ';' :  22,
  '<' :  23,
  '>' :  24,
  '?' :  25,
  'A' :  26,
  'B' :  27,
  'C' :  28,
  'D' :  29,
  'E' :  30,
  'F' :  31,
  'G' :  32,
  'H' :  33,
  'I' :  34,
  'J' :  35,
  'L' :  36,
  'M' :  37,
  'N' :  38,
  'O' :  39,
  'P' :  40,
  'Q' :  41,
  'R' :  42,
  'S' :  43,
  'T' :  44,
  'U' :  45,
  'V' :  46,
  'X' :  47,
  'Y' :  48,
  '^' :  49,
  '_' :  50,
  '`' :  51,
  'a' :  52,
  'b' :  53,
  'c' :  54,
  'd' :  55,
  'e' :  56,
  'f' :  57,
  'g' :  58,
  'h' :  59,
  'i' :  60,
  'j' :  61,
  'k' :  62,
  'l' :  63,
  'm' :  64,
  'n' :  65,
  'o' :  66,
  'p' :  67,
  'q' :  68,
  'r' :  69,
  's' :  70,
  't' :  71,
  'u' :  72,
  'v' :  73,
  'x' :  74,
  'y' :  75,
  'z' :  76,

Pasamos cada texto a un array de enteros

In [110]:
text_as_int = np.array([char2idx[c] for c in texto])

print ('texto : {}'.format(repr(texto[:50])))
print ('{}'.format(repr(texto[:50])))

texto : 'La Celestina {0}\n---------------------------------'
'La Celestina {0}\n---------------------------------'


### Preparación de los datos para entrenar la RNN

Para entrenar el modelo creamos un conjunto de datos con el contenido de text_as_init. Para ello utilizamos la función tf.data.Dataset.from_tensor_slices.
A este conjunto de datos lo dividiremos en secuencias de seq_length+1 al aplicar el método batch()


In [111]:
# Creamos una función `split_input_target` que devolverá el conjunto de datos
# de entrenamiento (los datos de entrada como los datos de salida)
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

#Agrupamos los dataset en batches de 64 .
# Así tendriamos los datos de entrenamiento con batches compuestos de 64 parejas
# de secuencias de 100 integers de 64 bits
BATCH_SIZE = 64
BUFFER_SIZE = 10000

In [112]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
seq_length = 100
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(10):
  print(repr(''.join(idx2char[item.numpy()])))

#Aplicamos split_input_target a todas las secuencias utilizando el método map()
dataset = sequences.map(split_input_target)

print("\n\n")

#Los dataset contienen un conjunto de parejas (100 caracteres del texto original, la correspondiente salida ). Vamos a mostrar la primera pareja.
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

  print(dataset)

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print (dataset)

'La Celestina {0}\n------------------------------------------------------------------------\n-----------'
'-------------------------------------------------------------\nTRAGICOMEDIA DE CALISTO Y MELIBEA: Text'
'o\n------------------------------------------------------------------------\n\n*SIGUESE*\n\nla comedia /o '
'tragicomedia / de calisto y melibea, compuesta en\nreprehension de los locos enamorados, que, vencidos'
' en su desordenado\napetito, a sus amigas llaman y dizen ser su Dios. Assi mesmo fecha en\nauiso de los'
' engaños de las alcahuetas y malos y lisonjeros siruientes.\n\n----------------------------------------'
'--------------------------------\n*ARGUMENTO GENERAL*\n\n*\n---------------------------------------------'
'---------------------------\nARGUMENTO de toda la obra\n\n*Calisto fue de noble linaje, de claro ingenio'
', de gentil disposición,\nde linda criança, dotado de muchas gracias, de estado mediano. Fue preso\nen '
'el amor de Melibea, {14"} muger moça, muy

### Construcción del modelo RNN

In [113]:
#Crearemos una función que cree un modelo RNN con tres capas
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, BatchNormalization

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = Sequential()
  #Añadimos la capa de tipo word embedding
  model.add(Embedding(input_dim=vocab_size,
                      output_dim=embedding_dim,
                      #batch_input_shape=[batch_size, None] Deprecated
                      ))
  #Añadimos la capa de tipo LSTM
  model.add(LSTM(rnn_units,
                 return_sequences=True,
                 stateful=True,
                 recurrent_initializer='glorot_uniform'))
  model.add(Dense(512, activation="relu"))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))


  #Añadimos la capa de tipo Dense
  model.add(Dense(vocab_size))
  return model

In [114]:
embedding_dim = 256
rnn_units = 1024

In [115]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_8 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [116]:
for input_example_batch, target_example_batch in dataset.take(1):
  print("Input:", input_example_batch.shape, "# (batch_size, sequence_length)")
  print("Target:", target_example_batch.shape, "# (batch_size, sequence_length)")

Input: (64, 100) # (batch_size, sequence_length)
Target: (64, 100) # (batch_size, sequence_length)


In [117]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print("Prediction : ", example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

Prediction :  (64, 100, 90) # (batch_size, sequence_length, vocab_size)


In [118]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices_characters = tf.squeeze(sampled_indices,axis=-1).numpy()

In [119]:
print(sampled_indices_characters)

[45 25 89 40 17 70 37 15 54 13 50 52 36  2 68 39  1 38  9 21  0 47 79  2
 49 67 62 51 29 14 83  8 83 28  5 31 58 80 71 41 23  6 28 87  5 68 18  1
  6 15 45 83  3 49 18 24 16 15 21 82 44 76 36 61 72 62 79 13 29 22 45 44
 50 68 82 66 27 83  0 14 36 87 20 81 85 19 73 69  4 25 43 75 89 81 39 15
 75 74 44 26]


### Entrenamiento del modelo RNN

In [120]:
#Creamos la función de perdida, usaremos el categorical pues estamos considerando datos categóricos
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [121]:
#Compilamos el modelo
model.compile(optimizer='adam', loss=loss)

In [122]:
#configuramos los checkpoints

checkpoint_dir = './training_checkpoints_Celestina'

# nombre fichero
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)


In [123]:
# Implementación EarlyStopping
from tensorflow.keras.callbacks import EarlyStopping

early_stopping_callback = EarlyStopping(
    monitor='loss',
    patience=10,
    min_delta=0.01,
    restore_best_weights=True
)

In [124]:
#Entrenamos el modelo
EPOCHS=300
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback, early_stopping_callback])

Epoch 1/300
58/58 ━━━━━━━━━━━━━━━━━━━━ 34s 573ms/step - loss: 3.1028
Epoch 2/300
58/58 ━━━━━━━━━━━━━━━━━━━━ 32s 555ms/step - loss: 2.1175
Epoch 3/300
58/58 ━━━━━━━━━━━━━━━━━━━━ 33s 564ms/step - loss: 1.9808
Epoch 4/300
58/58 ━━━━━━━━━━━━━━━━━━━━ 33s 570ms/step - loss: 1.8922
Epoch 5/300
58/58 ━━━━━━━━━━━━━━━━━━━━ 33s 570ms/step - loss: 1.8274
Epoch 6/300
58/58 ━━━━━━━━━━━━━━━━━━━━ 33s 569ms/step - loss: 1.7634
Epoch 7/300
58/58 ━━━━━━━━━━━━━━━━━━━━ 33s 568ms/step - loss: 1.6998
Epoch 8/300
58/58 ━━━━━━━━━━━━━━━━━━━━ 33s 570ms/step - loss: 1.6493
Epoch 9/300
58/58 ━━━━━━━━━━━━━━━━━━━━ 33s 566ms/step - loss: 1.5884
Epoch 10/300
58/58 ━━━━━━━━━━━━━━━━━━━━ 34s 575ms/step - loss: 1.5467
Epoch 11/300
58/58 ━━━━━━━━━━━━━━━━━━━━ 33s 571ms/step - loss: 1.5014
Epoch 12/300
58/58 ━━━━━━━━━━━━━━━━━━━━ 33s 568ms/step - loss: 1.4617
Epoch 13/300
58/58 ━━━━━━━━━━━━━━━━━━━━ 33s 569ms/step - loss: 1.4178
Epoch 14/300
58/58 ━━━━━━━━━━━━━━━━━━━━ 34s 588ms/step - loss: 1.3762
Epoch 15/300
58/58 ━━━━━━━━━━

In [125]:
model.save("model_celestina_100_2025.keras")

In [126]:
from keras.models import load_model
from keras import losses # Import the losses module

# Assuming your original loss function was, for example, binary_crossentropy
loaded_model = load_model("model_celestina_100_2025.keras",
                          custom_objects={'loss': losses.sparse_categorical_crossentropy})
# or if it was a custom loss function
# loaded_model = load_model("model_paquita_100_2024.keras", custom_objects={'loss': my_custom_loss_function})

In [127]:
model = build_model(len(vocab), embedding_dim, rnn_units, batch_size=1)
input_shape = (1, 100)  # Replace 100 with your actual sequence length
model.build(input_shape=input_shape) # Or model.build(tf.TensorShape([1, None]))

model.load_weights("model_celestina_100_2025.keras")

In [130]:
#Creamos una función generar_texto que generará texto a partir de una palabra de partida
def generate_text(model, start_string):

  num_generate = 1000
  input_eval = [char2idx[s] for s in start_string]

  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []


  # temperature = 0.7 # cuanto mas alto el numero, mas directa la salida de los logits
  temperature = 0.25 # con 0.3 va mejor segun testeo con la otra clase, con 0.1 se vuelve repetitivo

#  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)

      predictions = tf.squeeze(predictions, 0)

      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()


      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [131]:
print(generate_text(model, start_string=u"¿Quién eres tú, que así osas turbar mi sosiego en tan alta hora?"))

¿Quién eres tú, que así osas turbar mi sosiego en tan alta hora? assí
que con la les por el tiempo que perdí de no
gozarlo, de no conoscerlo, después que a mí me sé conoscer. No quiero
amar mi cara ? ¿ Qué hazes ? ¿
Porqué soy loca ? ¿ Porqué tengo fe con este couarde ? ¿ Porqué creo sus
mentadas ? ¡ O salud de mí ! Que se llamaua el mío, que en lor este fuego de Dios, no te destroces ni maltrates como sueles. ¿
Qué prouecho te trae dañar mis vestiduras ?
16. CAL. __ El te me recibirá algo de tan
alta en su pequeño y fazer las leyes de las cosas fue dizen:
33. SEMP. __ ¿ De qué te ríes ? ¡ De mal cancre sea comida essa boca
desgraciada, enojosa !
36. CEL. __ ¡ Hásteme el buen seso, viendo
la pérdida al ojo, viendo que los atauíos hazen la muger hermosa, avnque
no lo sepa: ¿ Hauíale yo de comenido ? ¿ A quién daré
parte de mi gloria ? Bien me dezía la vieja que de ninguna prosperidad
es bueno a mis passados y a mí, érades compañeros; mas, quando
el vil está rico, no tiene pariente ni am

# Intentos con notebook base

## Intento 1.1: 100 epocas y 0.3 temperatura
```
¿Quién eres tú, que así osas turbar mi sosiego en tan alta hora?
15. MELIB. __ Por Dios, perdido me tiempo no la primera lisión, que dio sobre
sano. Pues si tú quieres ser sana y que te descubra la punta de la cara. MELIB. __ Assí sea y vaya Dios contigo.
66. PLEB. __ Señora muger, ¿ Duermes ?
67. ALI. __ Señor, no pienses que traygo de aquella propia escriptura
{361"} Cota o Mena con su gran saber.
{362"} Jamás yo no vide en lengua romana,
{362"} No os lance Cupido sus tiros dorados.
ACTO I
```
### Observaciones
Las palabras son correctas, aunque algunas parezcan mal escritas (como "muger" o "traygo") son simplemente palabras escritas de la forma antigua. El estilo es coherente con el estilo de la Celestina. Los números (los cuales son ordenados en el texto original) saltan primero de 15 a 66, pero después de 66 a 67, lo cual muestra que el modelo podría estar mejor entrenado pero aún así encuentra coherencia.

## Intento 1.2: 100 epocas y 0.5 temperatura
```
¿Quién eres tú, que así osas turbar mi sosiego en tan alta hora? Andéles, por Dios, en tal tarridar ? Tente cierto se sacaría a haglar en el cuerpo, de asco de oyrte llamar aquella
gentil. ¡ Mirad quién gentil ! ! Jesú, Jesú ! ¡ Y qué bozos !
11. MELIB. __ Está noche.
37. MELIB. __ Gloriosa me será buena maestra destas
labores. Pues agora verás quánto por mi causa vales, quánto con ello te creo. Que todas aquellas cosas, cuya possessión no es agradable, más
vale poseellas, que esperallas. Aquí lugar de fauorecer y tu suaue vanar
por su buena habla de comer y acompañáuala;
tornado y tú filosofando.
No te espero más. Saquen vn cauallo. Límpienle mucho. Aprieten bien la
cincha perdido con el desastrado
Píramo y de la desdichada Tisbe !
17. SEMP. __ ¿ Qué cosa es ?
18. CAL. __ No se ces me puede ser. No aprenden los cursos naturales a rodeando su visto, si a ti te puedo parar dispuesto de aquélla
a quien vosotros seruís y yo adoro y, por más que trabajo noches y días,
no me vale nombre de libre, quando cautiuaste tu voluntad.
24. CAL. __ ¡ Palos querrá
```
### Observaciones
Con la temperatura elevada empieza a inventarse palabras, como "tarridar", "haglar" o "vn cauallo", las cuales no se encuentran en el texto original. Los números también saltan, con dos números (en este caso 17/18) coincidiendo igual que en el intento anterior. En general se nota que el texto varía más en coherencia.

## Intento 1.3: 100 epocas y 0.2 temperatura
```
¿Quién eres tú, que así osas turbar mi sosiego en tan alta hora?
15. MELIB. __ ¡ O por Dios, que me matas ! ¿ Y no me marauillo que le meten de matar a tus secretos mouimientos, para saber la secreta causa
de que proceden de su grado ? ! O Dios mio, qué alto don !
51. CEL. __ Pues más le pedí.
52. CAL. __ ¿ Qué, mi vieja ? ? Todas las viene las assentadas de viuir por tan bien con el baxar lo que en si no tiene orden ni
consejo ?
49. SEMP. __ ¡ Ha ! ! ha ! ! ha ! ¿ Esto es el fuego, diziendo que se arrima. . . Tarde fuy; pero temprano recabdé. ¡ O hermano ! ? Qué te
contaría de sus gracias en el proceder a guardar con este modo !
81. SEMP. __ No te marauilles, hija, que quien en muchas partes derrama
su memoria. Yo me voy. Cumple, señor, que si salieres mañana, lleues reboçado vn paño,
porque si della fueres por las puertas de
tu cara. está de amor que la ver y hablar, la habla
en nueua amanos de las puertas hazen con
azeytes vsar su vida, como de sus mismos padres. Con todos
tenía quehazer, con todos fablaua. Si salíamos por la calle, rogalla ? ¿ 
```
### Observaciones
En este caso, el modelo copia palabra a palabra varias frases directamente del texto original. Por ejemplo, "Yo me voy. Cumple, señor, que si salieres mañana, lleues reboçado vn paño" se encuentra así exactamente en el texto original de La Celestina. Esto sugiere overfitting.

## Conclusión 100 épocas
Vistos los tres ejemplos, cada uno con su temperatura, se puede ver que el más balanceado entre coherencia y repetitividad es el que tiene la temperatura en 0.3. Ambos son fieles al estilo del texto original. Aún así, incrementamos la cantidad de épocas para ver si el modelo podrá conseguir más coherencia, porque aunque tiene palabras y no copia frases del texto original, la sintaxis no es del todo correcta y podría tener más sentido lo que dice. 

## Intento 3.1: 300 épocas y 0.3 temperatura
```
¿Quién eres tú, que así osas turbar mi sosiego en tan alta hora? Que no hiziste seso de lo que pides ?
76. CEL. __ No dize, hija, sino que se huelga mucho con tu amistad,
porque eres más cierta se embueluas y con ello estés sin vn momento te partir, hasta que
Melibea con aparejada oportunidad que en los bienes de tu hermano y el vino hazen a los hombres renegar.
Conséjate con Séneca y verás en qué las tiene. Escucha al presente podrás ser seruida.
11. MELIB. __ ¡ O por Dios, no se cometa tal cosa ! Pero mucho plazer
tengo que de tan fiera hora la puerta de la virtud del río, en al es ser sano que poderlo ser, y mejor es poder ser doliente que
ser enfermo la mia mucho de nosotros manda que con él
mundo se oluida la mayor ni poderío en
mi vida que Dios.
189. PARM. __ ¿ Por qué, señor, te matas ? ¿ Por qué, señor, te contaré es no era ! y también sabes tú quánta más necessidad tienen
los viejos que los moços, mayormente tú que vas a mi y a la desdichada de tu padre, traydo la muerte al
couarde. ¡ O quántas mugeres
y el
plazer de la desdichada mia, los
```

### Observaciones
Sigue sin ser muy coherente, pero ha tardado 16 horas en entrenarse y por lo tanto no se seguirá entrenando más de esto.

## Intento 3.2: 300 épocas y 0.2 temperatura
```
¿Quién eres tú, que así osas turbar mi sosiego en tan alta hora? Que no ay mejor lo que deuerno a su amo haré reuessar
el plazer comido. y tú, Elicia, alma mia, no recibas pena. Passa a mi
casa a su propio padre.
62. CAL. __ En todo lo dicho es no haze hábito en que ay
clara con vn ebúrneo pensamiento todo lo que prometí, pues jamás
al esfuerço desayudó la fortuna. Ya veo su puerta llaman y vimismos. Pues más merecido de amor, que no me destrobles la mesa ! Al
te sabe en otra artas en el mundo yazeys mentiras, sus enemistades, con tus passadas fuerue a quebraré vn ojo que enojarte.
84. CEL. __ No tengo ya enojo; pero porque veya que le consejaua yo lo cierto y me daua malas
gracias. Pero de aquí del don nos en este tiempo las vía ha de yr este hecho ! No
basta loco, no podrás escapar, si siempre no te
acompaña quien te allegue plazeres, y la tornado de aquestos estremos, en que estoy perples no consientes como verdadera madre tuya, te digo, so
las malediciones, que tus padres te pusieron, si me fuesse mucho lo que jamás a
ti ni a otro pensé descobr
```
### Observaciones
Es más coherente que con 0.3 de temperatura, pero toma muchas frases directo del texto, igual que en el primer intento de 100 épocas con la misma temperatura. Esto sugiere overfitting.

## Intento 3.3: 300 épocas y 0.25 temperatura
```
¿Quién eres tú, que así osas turbar mi sosiego en tan alta hora? Que no hiziste seso allá será si lo has de salir amor, el principio
las culpas de la riqueza. ¡ Que por de tu padre. Pues a las manos me
has venido, donde te pueda dezir que es más cierto vso y costumbre, no
juzgues la bondad y hermosura de Melibea por esso ser la que la madre ¿ Fize bien ?
2. SEMP. __ ¡ Hay ! Si fiziste bien ! Allende se parescen, ladrillados por encima con
lisonjas. /Aquél es rico que está bien con Dios. Más segura con esfuerço, no lo he oluidado ni creas que he perdido
con los amores deste perdido
de nuestro amo. Quiero yrme al hilo de la gente, pues a los traydores llaman
discretos, a los fieles nescios. Si cuentas es te aprendescubre que que sujeción me relieua de culpa. No ayamos
enojo, assentémonos a comer.
17. ELIC. __ ¡ Assí ! ! Para esto más mal de merecer que aya vn año sería harto, no es mucha tu vida.
63. CAL. __ ¿ Quieres dezir que soy como el moço del escuerta de liberal. Después será. Procede en tu habla y dime qué
más passada y solicitud del principio
```
### Observaciones
Es el mejor entre los tres intentos, balancea originalidad y coherencia mejor que los dos intentos anteriores.

# Intentos con cambios
### EarlyStopping
Agregamos `EarlyStopping` por lo frecuente que es la repetición en el modelo de los intentos anteriores. Siguiendo con 300 épocas, el entrenamiento ahora frena temprano. Por ejemplo, en la primera ejecución con 300 épocas frena en la época 89 con una pérdida de 0.2089.
```
Epoch 89/300
58/58 ━━━━━━━━━━━━━━━━━━━━ 32s 552ms/step - loss: 0.2089
```
Esto indica que el modelo había dejado de mejorar significativamente y que continuar entrenando hubiera sido ineficiente y contraproducente por overfitting.
### BatchNormalization
Agregamos `BatchNormalization` porque estabiliza y acelera el entrenamiento al normalizar la salida de la capa `Dense`. Esto mejora la propagación del gradiente, reduce la sensibilidad a la inicialización de pesos y regulariza el modelo. También permite que la red entrene con mayor estabilidad al combinarse con activaciones como `ReLU.`
### Dropout
Bajamos el dropout porque el valor original (0.5) era demasiado agresivo y podía estar dificultando el aprendizaje de patrones. Al reducirlo a 0.3, se mantiene cierta regularización para prevenir el sobreajuste.

## Intentos con los cambios
## Intento 4.1: 300 épocas con early stop a las 89 épocas con pérdida de 0.2089.
```
¿Quién eres tú, que así osas turbar mi sosiego en tan alta hora? assí
que con la les por el tiempo que perdí de no
gozarlo, de no conoscerlo, después que a mí me sé conoscer. No quiero
amar mi cara ? ¿ Qué hazes ? ¿
Porqué soy loca ? ¿ Porqué tengo fe con este couarde ? ¿ Porqué creo sus
mentadas ? ¡ O salud de mí ! Que se llamaua el mío, que en lor este fuego de Dios, no te destroces ni maltrates como sueles. ¿
Qué prouecho te trae dañar mis vestiduras ?
16. CAL. __ El te me recibirá algo de tan
alta en su pequeño y fazer las leyes de las cosas fue dizen:
33. SEMP. __ ¿ De qué te ríes ? ¡ De mal cancre sea comida essa boca
desgraciada, enojosa !
36. CEL. __ ¡ Hásteme el buen seso, viendo
la pérdida al ojo, viendo que los atauíos hazen la muger hermosa, avnque
no lo sepa: ¿ Hauíale yo de comenido ? ¿ A quién daré
parte de mi gloria ? Bien me dezía la vieja que de ninguna prosperidad
es bueno a mis passados y a mí, érades compañeros; mas, quando
el vil está rico, no tiene pariente ni amigos ? Pues, loado Dios, bienes tienes. ¿ Y no sabes que has
men
```
### Observaciones
Esta versión del modelo genera texto más gramaticalmente coherente y estructuralmente aceptable. La fieldad al texto original sigue igual que en las versiones anteriores.

# Conclusión
Al principio pensé que los problemas podrían llegar a ser con la cantidad de entrenamiento que recibe el modelo, e incrementé de 100 a 300 épocas. Aún así el modelo no lograba formar frases coherentes, y al poner el EarlyStopping me dí cuenta que la gramática mejoraba bastante, no por la cantidad de épocas pero por la incorporación del BatchNormalization y la reduccio2n del Dropout.